In [1]:
using Pkg
Pkg.activate("..")

error: 'using' undefined near line 1 column 1
error: 'Pkg' undefined near line 1 column 1


## Library downloads

In [2]:
using JuMP,Ipopt,MosekTools, PyPlot
ipopt = () -> Ipopt.Optimizer()
mosek = () -> Mosek.Optimizer()

error: 'using' undefined near line 1 column 1
parse error:

  syntax error

>>> ipopt = () -> Ipopt.Optimizer()
             ^

parse error:

  syntax error

>>> mosek = () -> Mosek.Optimizer()
             ^



## Data Structure and auxiliary functions

In [3]:
include("../src/structs.jl")
include("../src/data_read.jl")
include("../src/utils.jl")

error: 'include' undefined near line 1 column 1
error: 'include' undefined near line 1 column 1
error: 'include' undefined near line 1 column 1


## Data Download

Data library: [pglib-opf-master](https://github.com/power-grid-lib/pglib-opf)

In [4]:
full_data = raw"..\data\pglib-opf"

parse error:

  syntax error

>>> full_data = raw"..\data\pglib-opf"
                                     ^



# Model build

## Functions related to electrical grid graph

Consider a non-directed graph with $\mathcal{N} = (\mathcal{B},\mathcal{L})$, where $\mathcal{B}$ is the set of buses and $\mathcal{L}$ the set of transmission lines. The set of Generators $\mathcal{G}$ is a subset of $\mathcal{B}$.

In [5]:
function bus_id(Buses)
    count = 1
    dict_buses = Dict()
    for bus in Buses
        dict_buses[bus.bus_i] = count
        count += 1
    end
    dict_buses
end

parse error:

  syntax error

>>>     for bus in Buses
                 ^

parse error:

  syntax error

>>>         dict_buses[bus.bus_i] = count
                      ^


Please read <https://www.octave.org/missing.html> to learn how you can
contribute missing functionality.
error: 'count' undefined near line 1 column 9
parse error:

  syntax error

>>>     end
          ^

error: 'dict_buses' undefined near line 1 column 5
parse error:

  syntax error

>>> end
      ^



The function $\delta$ receive a bus as attribute and returns all lines connected to it.

In [6]:
function δ(i,Branches,Buses)
    tbuses = []
    dict_buses = bus_id(Buses)
    count = 1
    ls = []
    for branch in Branches
        if dict_buses[branch.fbus] == i
            push!(tbuses,dict_buses[branch.tbus])
            push!(ls,count)
        end
        if dict_buses[branch.tbus] == i
            push!(tbuses,dict_buses[branch.fbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

parse error:

  syntax error

>>> function δ(i,Branches,Buses)
             ^

tbuses = [](0x0)
error: 'Buses' undefined near line 1 column 25
count =  1
ls = [](0x0)
parse error:

  syntax error

>>>     for branch in Branches
                    ^

parse error:

  syntax error

>>>             push!(tbuses,dict_buses[branch.tbus])
                    ^

parse error:

  syntax error

>>>             push!(ls,count)
                    ^

parse error:

  syntax error

>>>         end
              ^

parse error:

  syntax error

>>>             push!(tbuses,dict_buses[branch.fbus])
                    ^

parse error:

  syntax error

>>>             push!(ls,count)
                    ^

parse error:

  syntax error

>>>         end
              ^

count =  2
parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>     (tbuses,ls)
               ^

parse error:

  syntax error

>>> end
      ^



In [7]:
function From_nodes(i,Branches,Buses)
    tbuses = []
    ls = []
    count = 1
    dict_buses = bus_id(Buses)
    for branch in Branches
        if dict_buses[branch.fbus] == i
            push!(tbuses,dict_buses[branch.tbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

function To_nodes(i,Branches,Buses)
    tbuses = []
    ls = []
    count = 1
    dict_buses = bus_id(Buses)
    for branch in Branches
        if dict_buses[branch.tbus] == i
            push!(tbuses,dict_buses[branch.fbus])
            push!(ls,count)
        end
        count += 1
    end
    (tbuses,ls)
end

parse error:

  syntax error

>>>     for branch in Branches
                    ^

parse error:

  syntax error

>>>             push!(tbuses,dict_buses[branch.tbus])
                    ^

parse error:

  syntax error

>>>             push!(ls,count)
                    ^

parse error:

  syntax error

>>>         end
              ^

count =  3
parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>     (tbuses,ls)
               ^

parse error:

  syntax error

>>> end
      ^

parse error:

  syntax error

>>>     for branch in Branches
                    ^

parse error:

  syntax error

>>>             push!(tbuses,dict_buses[branch.fbus])
                    ^

parse error:

  syntax error

>>>             push!(ls,count)
                    ^

parse error:

  syntax error

>>>         end
              ^

count =  4
parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>     (tbuses,ls)
               ^

parse e

In [8]:
function G(i,Generators,Buses)
    G_i = []
    count = 1
    dict_buses = bus_id(Buses)
    for gen in Generators
        if dict_buses[gen.bus] == i
            push!(G_i,count)
        end
        count += 1
    end
    G_i
end

parse error:

  syntax error

>>>     for gen in Generators
                 ^

parse error:

  syntax error

>>>             push!(G_i,count)
                    ^

parse error:

  syntax error

>>>         end
              ^

count =  5
parse error:

  syntax error

>>>     end
          ^

error: 'G_i' undefined near line 1 column 5
parse error:

  syntax error

>>> end
      ^



## Constants

- $Y_l^{Re}$: Series admittance (real)
- $Y_l^{Im}$: Series admittance (imaginary)
- $T_l^{Re}$: Transformer parameters (real)
- $T_l^{Im}$: Transformer parameters (imaginary)
- $b_l$: Line charge
- $s_l^u$: Thermal limits
- $\theta_{min}$,$\theta_{max}$: Branch voltage angle difference range

In [9]:
function add_constants(Branches)
    YR = zeros(length(Branches))
    YI = zeros(length(Branches))
    TR = zeros(length(Branches))
    TI = zeros(length(Branches))
    B = zeros(length(Branches))
    thermal_lim = zeros(length(Branches))
    ang_min = zeros(length(Branches))
    ang_max = zeros(length(Branches))
    for l=1:length(Branches)
        YR[l] = Branches[l].r/(Branches[l].r^2+Branches[l].x^2)
        YI[l] = -Branches[l].x/(Branches[l].r^2+Branches[l].x^2)
        TR[l] = Branches[l].ratio*cos(Branches[l].angle)
        TI[l] = Branches[l].ratio*sin(Branches[l].angle)
        B[l] = Branches[l].b
        thermal_lim[l] = Branches[l].rateA/100.0
        ang_min[l] = Branches[l].angmin*pi/180
        ang_max[l] = Branches[l].angmax*pi/180
    end
    return YR,YI,TR,TI,B,thermal_lim,ang_min,ang_max
end

parse error:

  syntax error

>>>         YR[l] = Branches[l].r/(Branches[l].r^2+Branches[l].x^2)
              ^

parse error:

  syntax error

>>>         YI[l] = -Branches[l].x/(Branches[l].r^2+Branches[l].x^2)
              ^

parse error:

  syntax error

>>>         TR[l] = Branches[l].ratio*cos(Branches[l].angle)
              ^

parse error:

  syntax error

>>>         TI[l] = Branches[l].ratio*sin(Branches[l].angle)
              ^

parse error:

  syntax error

>>>         B[l] = Branches[l].b
             ^

parse error:

  syntax error

>>>         thermal_lim[l] = Branches[l].rateA/100.0
                       ^

parse error:

  syntax error

>>>         ang_min[l] = Branches[l].angmin*pi/180
                   ^

parse error:

  syntax error

>>>         ang_max[l] = Branches[l].angmax*pi/180
                   ^

parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>     return YR,YI,TR,TI,B,thermal_lim,ang_min,ang_max
                ^

## Unrelaxed variables

- $p_g^i \hspace{0.2cm} \forall i \in \mathcal{B}$: Power Output (active)
- $q_g^i \hspace{0.2cm} \forall i \in \mathcal{B}$: Power Output (reactive)
- $S^{Re}_{i,j,l} \hspace{0.2cm} \forall (i,j,l) \in \mathcal{L}$: Power flow on line (active)
- $S^{Im}_{i,j,l} \hspace{0.2cm} \forall (i,j,l) \in \mathcal{L}$: Power flow on line (reactive)

In [10]:
function add_global_variables!(model, Buses, Generators, Branches, thermal_lim; start=false)
    if start
        arr1 = ones(length(Buses))
        arr2 = ones(length(Generators))
        arr3 = ones(length(Branches))
        @variable(model, Generators[i].Pmin/100.0 <= p_g[i=1:length(Generators)] <= Generators[i].Pmax/100.0, start = arr2[i])
        @variable(model, Generators[i].Qmin/100.0 <= q_g[i=1:length(Generators)] <= Generators[i].Qmax/100.0, start = arr2[i])
        SR = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SR",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l], start = arr3[l])
        SI = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SI",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l], start = arr3[l])
    else
        @variable(model, Generators[i].Pmin/100.0 <= p_g[i=1:length(Generators)] <= Generators[i].Pmax/100.0)
        @variable(model, Generators[i].Qmin/100.0 <= q_g[i=1:length(Generators)] <= Generators[i].Qmax/100.0)
        SR = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SR",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l])
        SI = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SI",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l])
    end
    model[:SR] = SR
    model[:SI] = SI
end

parse error:

  syntax error

>>> function add_global_variables!(model, Buses, Generators, Branches, thermal_lim; start=false)
                                        ^

parse error:

  syntax error

>>>         @variable(model, Generators[i].Pmin/100.0 <= p_g[i=1:length(Generators)] <= Generators[i].Pmax/100.0, start = arr2[i])
                                       ^

parse error:

  syntax error

>>>         @variable(model, Generators[i].Qmin/100.0 <= q_g[i=1:length(Generators)] <= Generators[i].Qmax/100.0, start = arr2[i])
                                       ^

parse error:

  syntax error

>>>         SR = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,Buses)...)]], base_name = "SR",lower_bound = -thermal_lim[l], upper_bound = thermal_lim[l], start = arr3[l])
                                    ^

parse error:

  syntax error

>>>         SI = @variable(model,[(i,j,l) in [(i,j,l) for i = 1:length(Buses) for (j,l) in zip(δ(i,Branches,

## Voltage Variables

- $V_i^{Re} \hspace{0.2cm} \forall i \in \mathcal{B}$: Bus Voltage (active)
- $V_i^{Im} \hspace{0.2cm} \forall i \in \mathcal{B}$: Bus Voltage (reactive)

In [11]:
function add_voltage_variables!(model, Buses; start = false)
    if start
        arr1 = ones(length(Buses))
        @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
        @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
    else
        @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax)
        @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax)
    end
end

parse error:

  syntax error

>>> function add_voltage_variables!(model, Buses; start = false)
                                         ^

parse error:

  syntax error

>>>         @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
                                    ^

parse error:

  syntax error

>>>         @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax , start = arr1[i])
                                    ^

parse error:

  syntax error

>>>     else
           ^

parse error:

  syntax error

>>>         @variable(model, - Buses[i].Vmax <= VR[i=1:length(Buses)] <= Buses[i].Vmax)
                                    ^

parse error:

  syntax error

>>>         @variable(model, - Buses[i].Vmax <= VI[i=1:length(Buses)] <= Buses[i].Vmax)
                                    ^

parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>> end
      ^



## Constraints

### Reference angle
- Constraint:
    - $\angle V_I(1) = 0$

In [12]:
function add_reference_angle!(model, Buses)
    VI = variable_by_name(model, "VI[1]")
    @constraint(model, VI == 0)
end

parse error:

  syntax error

>>> function add_reference_angle!(model, Buses)
                                       ^

error: 'model' undefined near line 1 column 27
error: @constraint: no function and no method found
parse error:

  syntax error

>>> end
      ^



### Power Balance
- Real and Imaginary part of a complex number: $\Re(\cdot),\Im(\cdot)$ 
- Constant Power Demand on bus i (active)  : $P^d_i$
- Constant Power Demand on bus i (reactive): $Q^d_i$
- Bus Shunt Admittance: $Y^s_i$
- Constraints:
    - $\sum_{i \in G(i)} p_g^i - P^d_i + \Re(Y^s_i) |V_i|^2 = \sum_{(l,j) \in \delta(i)} S_{lij}^{Re} \hspace{0.2cm}  \forall i \in \mathcal{B}$ (active)
    - $\sum_{i \in G(i)} q_g^i - Q^d_i + \Im(Y^s_i) |V_i|^2 = \sum_{(l,j) \in \delta(i)} S_{lij}^{Re} \hspace{0.2cm}  \forall i \in \mathcal{B}$ (reactive)

In [13]:
function constraint_power_balance!(model, Buses, Generators, Branches, sqrd_volt, i)
    p_g = model[Symbol("p_g")]
    q_g = model[Symbol("q_g")]
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    @constraint(model, sum(p_g[i] for i in G(i,Generators,Buses)) - Buses[i].Pd/100.0 + Buses[i].Gs/100.0*sqrd_volt == sum(SR[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
    @constraint(model, sum(q_g[i] for i in G(i,Generators,Buses)) - Buses[i].Qd/100.0 + Buses[i].Bs/100.0*sqrd_volt == sum(SI[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
end

parse error:

  syntax error

>>> function constraint_power_balance!(model, Buses, Generators, Branches, sqrd_volt, i)
                                            ^

parse error:

  syntax error

>>>     p_g = model[Symbol("p_g")]
                   ^

parse error:

  syntax error

>>>     q_g = model[Symbol("q_g")]
                   ^

parse error:

  syntax error

>>>     SR = model[Symbol("SR")]
                  ^

parse error:

  syntax error

>>>     SI = model[Symbol("SI")]
                  ^

parse error:

  syntax error

>>>     @constraint(model, sum(p_g[i] for i in G(i,Generators,Buses)) - Buses[i].Pd/100.0 + Buses[i].Gs/100.0*sqrd_volt == sum(SR[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
                                  ^

parse error:

  syntax error

>>>     @constraint(model, sum(q_g[i] for i in G(i,Generators,Buses)) - Buses[i].Qd/100.0 + Buses[i].Bs/100.0*sqrd_volt == sum(SI[(i,j,l)] for (j,l) in zip(δ(i,Branches,Buses)...)))
                               

### Voltage Bounds
- $v_i^l,v_i^u$: Lower and upper voltages limits
- Constraints:
    - $(v_i^l)^2 \leq |V_i|^2 \leq (v_i^u)^2 \hspace{0.2cm} \forall i \in \mathcal{B}$

In [14]:
function voltage_bounds_sqrd!(model, Buses, sqrd_volt, i)
    @constraint(model, Buses[i].Vmin^2 <= sqrd_volt <= Buses[i].Vmax^2)
end

parse error:

  syntax error

>>> function voltage_bounds_sqrd!(model, Buses, sqrd_volt, i)
                                       ^

parse error:

  syntax error

>>>     @constraint(model, Buses[i].Vmin^2 <= sqrd_volt <= Buses[i].Vmax^2)
                                ^

parse error:

  syntax error

>>> end
      ^



### Line Balance
- Constraints:
    - $S_{lij}^{Re} = Y_l^{Re} |V_i|^2 - Y_l^{Re}c_{ij} - Y_l^{Im}s_{ij} \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$
    - $S_{lij}^{Im} = -(Y_l^{Im}-b_l/2) |V_i|^2 + Y_l^{Im}c_{ij} - Y_l^{Re}s_{ij} \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$
    - $c_{ij} = V_i^{Re}V_j^{Re}+V_i^{Im}V_j^{Im}$
	- $s_{ij} = V_i^{Im}V_j^{Re}-V_i^{Re}V_j^{Im}$

In [15]:
function constraint_line_balance!(model, YR, YI, B, sqrd_volt_i, sqrd_volt_j, sum_product_voltages,  diff_product_voltages, i, j, l)
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    
    @constraint(model, SR[(i,j,l)] == YR*sqrd_volt_i + (-YR)*sum_product_voltages + (-YI)*(diff_product_voltages))
            
    @constraint(model, SI[(i,j,l)] == -(YI+B/2)*sqrd_volt_i - (-YI)*sum_product_voltages + (-YR)*diff_product_voltages)

    @constraint(model, SR[(j,i,l)] == YR*sqrd_volt_j + (-YR)*sum_product_voltages + (-YI)*(-(diff_product_voltages)))

    @constraint(model, SI[(j,i,l)] == -(YI+B/2)*sqrd_volt_j - (-YI)*sum_product_voltages + (-YR)*(-(diff_product_voltages)))
end

parse error:

  syntax error

>>> function constraint_line_balance!(model, YR, YI, B, sqrd_volt_i, sqrd_volt_j, sum_product_voltages,  diff_product_voltages, i, j, l)
                                           ^

parse error:

  syntax error

>>>     SR = model[Symbol("SR")]
                  ^

parse error:

  syntax error

>>>     SI = model[Symbol("SI")]
                  ^

parse error:

  syntax error

>>>     @constraint(model, SR[(i,j,l)] == YR*sqrd_volt_i + (-YR)*sum_product_voltages + (-YI)*(diff_product_voltages))
                             ^

parse error:

  syntax error

>>>     @constraint(model, SI[(i,j,l)] == -(YI+B/2)*sqrd_volt_i - (-YI)*sum_product_voltages + (-YR)*diff_product_voltages)
                             ^

parse error:

  syntax error

>>>     @constraint(model, SR[(j,i,l)] == YR*sqrd_volt_j + (-YR)*sum_product_voltages + (-YI)*(-(diff_product_voltages)))
                             ^

parse error:

  syntax error

>>>     @constraint(model, SI[(j,i,l)]

### Line thermal limit

- Constraints:
    - $(S_{lij}^{Re})^2 + (S_{lij}^{Im})^2 \leq (s_l^u)^2 \hspace{0.2cm} \forall (l,i,j) \in \mathcal{L}$

In [16]:
function line_thermal_bounds_sqrd!(model,thermal_lim, i, j, l)
    SR = model[Symbol("SR")]
    SI = model[Symbol("SI")]
    
    @constraint(model, SI[(i,j,l)]^2+SR[(i,j,l)]^2 <= thermal_lim[l]^2)
    @constraint(model, SI[(j,i,l)]^2+SR[(j,i,l)]^2 <= thermal_lim[l]^2)
end

parse error:

  syntax error

>>> function line_thermal_bounds_sqrd!(model,thermal_lim, i, j, l)
                                            ^

parse error:

  syntax error

>>>     SR = model[Symbol("SR")]
                  ^

parse error:

  syntax error

>>>     SI = model[Symbol("SI")]
                  ^

parse error:

  syntax error

>>>     @constraint(model, SI[(i,j,l)]^2+SR[(i,j,l)]^2 <= thermal_lim[l]^2)
                             ^

parse error:

  syntax error

>>>     @constraint(model, SI[(j,i,l)]^2+SR[(j,i,l)]^2 <= thermal_lim[l]^2)
                             ^

parse error:

  syntax error

>>> end
      ^



### Phase Angle Differences

- Constraints:
    - $ c_{ij} \tan(\theta_{min}) \leq s_{ij} \leq c_{ij} \tan(\theta_{max})$
    - $c_{ij} = V_i^{Re}V_j^{Re}+V_i^{Im}V_j^{Im}$
	- $s_{ij} = V_i^{Im}V_j^{Re}-V_i^{Re}V_j^{Im}$

In [17]:
function constraint_phase_angle_diff!(model, ang_max, ang_min, sum_product_voltages,  diff_product_voltages, i, j, l) 
    @constraint(model, diff_product_voltages <= tan(ang_max[l])*sum_product_voltages)
    @constraint(model, diff_product_voltages >= tan(ang_min[l])*sum_product_voltages)
end

parse error:

  syntax error

>>> function constraint_phase_angle_diff!(model, ang_max, ang_min, sum_product_voltages,  diff_product_voltages, i, j, l) 
                                               ^

parse error:

  syntax error

>>>     @constraint(model, diff_product_voltages <= tan(ang_max[l])*sum_product_voltages)
                                                               ^

parse error:

  syntax error

>>>     @constraint(model, diff_product_voltages >= tan(ang_min[l])*sum_product_voltages)
                                                               ^

parse error:

  syntax error

>>> end
      ^



## Objective Function

### Generator fuel cost minimization
- $\min \sum_{k \in \mathcal{G}} c_{2k} (p_k^g)^2 + c_{1k} p_k^g + c_{0k}$

In [18]:
function add_objective!(model, GeneratorCosts)
    p_g = model[Symbol("p_g")]
    q_g = model[Symbol("q_g")]
    obj = sum([GeneratorCosts[k].c2*p_g[k]^2*100.0^2+GeneratorCosts[k].c1*p_g[k]*100.0+GeneratorCosts[k].c0 for k = 1:length(GeneratorCosts)])
    @objective(model, Min, obj)
end

parse error:

  syntax error

>>> function add_objective!(model, GeneratorCosts)
                                 ^

parse error:

  syntax error

>>>     p_g = model[Symbol("p_g")]
                   ^

parse error:

  syntax error

>>>     q_g = model[Symbol("q_g")]
                   ^

parse error:

  syntax error

>>>     obj = sum([GeneratorCosts[k].c2*p_g[k]^2*100.0^2+GeneratorCosts[k].c1*p_g[k]*100.0+GeneratorCosts[k].c0 for k = 1:length(GeneratorCosts)])
                                 ^

error: @objective: no function and no method found
parse error:

  syntax error

>>> end
      ^



# Convex Relaxations

Consider the complex matrix $X \in \mathbb{R}^{\mathcal{|B|}\times \mathcal{|B|}}$
- $X = V(V^T)^*$
- $X_{ij} = V_iV_j^* \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

$X$ is positive semidefinite matrix with rank $1$.

We can substitute the complex matrix to a real matrix $W \in \mathbb{R}^{2\mathcal{|B|}\times 2 \mathcal{|B|}}$, with the following transformations:

- $W_{ij}   = V_i^{Re}V_j^{Re} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{i'j'} = V_i^{Im}V_j^{Im} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{i'j}  = V_i^{Im}V_j^{Re} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $W_{ij'}  = V_i^{Re}V_j^{Im} \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

Where $i' = i + |\mathcal{B}|$ and $j' = i + |\mathcal{B}|$.

$W$ is positive semidefinite matrix with rank $1$.

## McCormick Envelopes

The McCormick envelopes are the convex hull of the set $\{(x,y,w) \hspace{0.1cm} | \hspace{0.1cm} w = xy, (x,y) \in [\underline{x}, \bar{x}] \times [\underline{y}, \bar{y}]\}$, which are:
- $MC(w = xy) = \{(x, y, w): \max \{\underline{y} x+\underline{x} y-\underline{x} \underline{y}, \bar{y} x+\bar{x} y-\overline{x y}\} \leq w \leq \min \{\underline{y} x+\bar{x} y-\bar{x} \underline{y}, \bar{y} x+\underline{x} y-\underline{x} \bar{y}\}\}$

### Function to make McCormick envelopes

In [19]:
function MC_Envelopes!(model,w,x,y)
    x_up = upper_bound(x) 
    x_down = lower_bound(x)
    y_up = upper_bound(y) 
    y_down = lower_bound(y)
    @constraint(model, y_down*x+x_down*y-x_down*y_down <= w)
    @constraint(model, w <= y_down*x+x_up*y-x_up*y_down)
    @constraint(model, y_up*x+x_up*y-x_up*y_up <= w)
    @constraint(model, w <= y_up*x+x_down*y-x_down*y_up)
end

parse error:

  syntax error

>>> function MC_Envelopes!(model,w,x,y)
                                ^

error: 'x' undefined near line 1 column 24
error: 'x' undefined near line 1 column 26
error: 'y' undefined near line 1 column 24
error: 'y' undefined near line 1 column 26
error: @constraint: no function and no method found
error: @constraint: no function and no method found
error: @constraint: no function and no method found
error: @constraint: no function and no method found
parse error:

  syntax error

>>> end
      ^



### Relaxation with McCormick Envelopes

- $MC(W_{ij}   = V_i^{Re}V_j^{Re}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{i'j'} = V_i^{Im}V_j^{Im}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{i'j}  = V_i^{Im}V_j^{Re}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$
- $MC(W_{ij'}  = V_i^{Re}V_j^{Im}) \hspace{0.2cm} \forall (i,j) \in \mathcal{B}^2$

In [20]:
function MC_relaxation!(model,Buses,Branches)
    VR = model[Symbol("VR")]
    VI = model[Symbol("VI")]
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
    s = x -> x+length(Buses)
    sqrd_volt = [W[i,i]+W[s(i),s(i)] for i=1:length(Buses)]
    for i=1:length(Buses)
        MC_Envelopes!(model, W[i,i], VR[i], VR[i])
        MC_Envelopes!(model, W[s(i),s(i)], VI[i], VI[i])
    end
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)    
            MC_Envelopes!(model,W[i,j]      ,VR[i],VR[j])
            MC_Envelopes!(model,W[s(i),s(j)],VI[i],VI[j])
            MC_Envelopes!(model,W[s(i),j]   ,VI[i],VR[j])
            MC_Envelopes!(model,W[j,i]      ,VR[j],VR[i])
            MC_Envelopes!(model,W[s(j),s(i)],VI[j],VI[i])
            MC_Envelopes!(model,W[j,s(i)]   ,VR[i],VI[j])
        end
    end
end

parse error:

  syntax error

>>> function MC_relaxation!(model,Buses,Branches)
                                 ^

parse error:

  syntax error

>>>     VR = model[Symbol("VR")]
                  ^

parse error:

  syntax error

>>>     VI = model[Symbol("VI")]
                  ^

parse error:

  syntax error

>>>     @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
                          ^

parse error:

  syntax error

>>>     s = x -> x+length(Buses)
               ^

parse error:

  syntax error

>>>     sqrd_volt = [W[i,i]+W[s(i),s(i)] for i=1:length(Buses)]
                      ^

parse error:

  syntax error

>>>         MC_Envelopes!(model, W[i,i], VR[i], VR[i])
                        ^

parse error:

  syntax error

>>>         MC_Envelopes!(model, W[s(i),s(i)], VI[i], VI[i])
                        ^

parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>         for (j,l) in zip(From_nodes(i,Branches,Buses)...)    
       

## Semidefinite Relaxation

- The rank $1$ restriction of $W$ matrix is non-convex, so we remove this constraint.

In [21]:
function SDP_relaxation!(model,Buses)
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)],PSD)
end

parse error:

  syntax error

>>> function SDP_relaxation!(model,Buses)
                                  ^

parse error:

  syntax error

>>>     @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)],PSD)
                          ^

parse error:

  syntax error

>>> end
      ^



## Second Order Cone Relaxation

The SOCP relaxation consists in relaxing the equalitry $X = V(V^*)^T$ to an inequality

$\hspace{0.5cm} \begin{array}{l}
	X_{i j}=V_{i} V_{j}^{*} \\
	X_{i j} X_{i j}^{*}=V_{i} V_{j}^{*} V_{i}^{*} V_{j} \\
	\left|X_{i j}\right|^{2}=X_{i i} X_{j j} \\
	\left|X_{i j}\right|^{2} \leqslant X_{i i} X_{j j}
	\end{array}$

Writing as a second order cone:

$\hspace{0.5cm} \left|\left(\begin{array}{c}
		2 X_{i j} \\
		X_{i i}-X_{j j}
		\end{array}\right) \right| \leqslant X_{i i}+X_{j j}$
        
In function of $W$:

$\hspace{0.5cm} \left|\left(\begin{array}{c}
2(W_{i j}+W_{i' j'}) \\
W_{i i}+W_{i' i'}-W_{j j}-W_{j' j'}
\end{array}\right) \right| \leqslant W_{i i}+W_{i' i'}+W_{j j}+W_{j' j'}$

In [22]:
function SOCP_relaxation!(model,Buses,Branches)
    @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
    s = x -> x+length(Buses)
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)
            @constraint(model, [W[i,i]+W[s(i),s(i)]+W[j,j]+W[s(j),s(j)], 2*(W[i,j]+W[s(i),s(j)]), W[i,i]+W[s(i),s(i)]-W[j,j]-W[s(j),s(j)]] in SecondOrderCone())
        end
    end
end

parse error:

  syntax error

>>> function SOCP_relaxation!(model,Buses,Branches)
                                   ^

parse error:

  syntax error

>>>     @variable(model, W[i=1:2*length(Buses),j=1:2*length(Buses)])
                          ^

parse error:

  syntax error

>>>     s = x -> x+length(Buses)
               ^

parse error:

  syntax error

>>>         for (j,l) in zip(From_nodes(i,Branches,Buses)...)
                  ^

parse error:

  syntax error

>>>             @constraint(model, [W[i,i]+W[s(i),s(i)]+W[j,j]+W[s(j),s(j)], 2*(W[i,j]+W[s(i),s(j)]), W[i,i]+W[s(i),s(i)]-W[j,j]-W[s(j),s(j)]] in SecondOrderCone())
                                     ^

parse error:

  syntax error

>>>         end
              ^

parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>> end
      ^



# Model

In [23]:
function build_problem(file::String, optimizer, relaxation::RELAXATION)
    Buses, Generators, GeneratorCosts, Branches = read_matlab_file(file);
    YR,YI,TR,TI,B,thermal_lim,ang_min,ang_max = add_constants(Branches)
    model = Model(optimizer)
    if relaxation == NL
        add_global_variables!(model, Buses, Generators, Branches, thermal_lim, start = true)
    else
        add_global_variables!(model, Buses, Generators, Branches, thermal_lim)
    end
    add_objective!(model, GeneratorCosts)
    
    if relaxation == NL 
        add_voltage_variables!(model, Buses, start = true)
    elseif relaxation == MC
        add_voltage_variables!(model, Buses)
    end
    if relaxation == NL || relaxation == MC
        add_reference_angle!(model, Buses)
        VR = model[Symbol("VR")]
        VI = model[Symbol("VI")]
    end
    
    if relaxation == NL
        sqrd_volt = VR.^2 .+ VI.^2
    end
    
        
    if relaxation == MC
        MC_relaxation!(model,Buses,Branches)
    end
    if relaxation == SDP
        SDP_relaxation!(model,Buses)
    end
    if relaxation == SOCP
        SOCP_relaxation!(model,Buses,Branches)
    end
    
    if relaxation == MC || relaxation == SDP || relaxation == SOCP
        W = model[Symbol("W")]
        s = x -> x+length(Buses)
        sqrd_volt = [W[i,i]+W[s(i),s(i)] for i=1:length(Buses)]
    end
    
    for i=1:length(Buses)
        constraint_power_balance!(model, Buses, Generators, Branches, sqrd_volt[i], i)
        voltage_bounds_sqrd!(model, Buses, sqrd_volt[i], i)
    end
    
    for i=1:length(Buses)
        for (j,l) in zip(From_nodes(i,Branches,Buses)...)
            if relaxation == NL
                sum_product_voltages  = VR[i]*VR[j]+VI[i]*VI[j]
                diff_product_voltages = VI[i]*VR[j]-VR[i]*VI[j]
            elseif relaxation == MC || relaxation == SDP || relaxation == SOCP
                sum_product_voltages  = W[i,j]+W[s(i),s(j)]
                diff_product_voltages = W[s(j),i]-W[s(i),j]
            end
            constraint_line_balance!(model, YR[l], YI[l], B[l], sqrd_volt[i], sqrd_volt[j], sum_product_voltages,  diff_product_voltages, i, j, l)
            constraint_phase_angle_diff!(model, ang_max, ang_min, sum_product_voltages, diff_product_voltages, i, j, l)
            line_thermal_bounds_sqrd!(model,thermal_lim, i, j, l)
        end
    end
    return model
end

parse error:

  invalid parameter list

>>> function build_problem(file::String, optimizer, relaxation::RELAXATION)
                               ^

error: 'Buses' undefined near line 1 column 5
error: 'YR' undefined near line 1 column 5
error: 'optimizer' undefined near line 1 column 19
parse error:

  syntax error

>>>         add_global_variables!(model, Buses, Generators, Branches, thermal_lim, start = true)
                                ^

parse error:

  syntax error

>>>     else
           ^

parse error:

  syntax error

>>>         add_global_variables!(model, Buses, Generators, Branches, thermal_lim)
                                ^

parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>>     add_objective!(model, GeneratorCosts)
                     ^

parse error:

  syntax error

>>>         add_voltage_variables!(model, Buses, start = true)
                                 ^

parse error:

  syntax error

>>>     elseif relaxation == 

# Tests

## 5 buses

$|\mathcal{B}| = 5 $

$|\mathcal{L}| = 6 $

$|\mathcal{G}| = 5 $

In [24]:
case_test = case_tests[2]

parse error:

  syntax error

>>> case_test = case_tests[2]
                          ^



In [25]:
optimal_value = case_test[2][1]
filepath = joinpath(full_data,case_test[1]*".m")

parse error:

  syntax error

>>> optimal_value = case_test[2][1]
                             ^

parse error:

  syntax error

>>> filepath = joinpath(full_data,case_test[1]*".m")
                                           ^



### Non linear - IPOPT

In [26]:
model_nl = build_problem(filepath,ipopt,NL);

error: 'filepath' undefined near line 1 column 26


In [27]:
optimize!(model_nl)

parse error:

  syntax error

>>> optimize!(model_nl)
            ^



In [28]:
optimality_gap(model_nl,optimal_value)

error: 'model_nl' undefined near line 1 column 16


### McCormick Relaxation - MOSEK

In [29]:
model_mc = build_problem(filepath,mosek,MC);

error: 'filepath' undefined near line 1 column 26


In [30]:
optimize!(model_mc)

parse error:

  syntax error

>>> optimize!(model_mc)
            ^



In [31]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

error: 'model_mc' undefined near line 1 column 22


### Semidefinite Relaxation - MOSEK

In [32]:
model_sdp = build_problem(filepath,mosek,SDP);

error: 'filepath' undefined near line 1 column 27


In [33]:
optimize!(model_sdp)

parse error:

  syntax error

>>> optimize!(model_sdp)
            ^



In [34]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

error: 'model_sdp' undefined near line 1 column 22


### Second Order Cone Relaxation - MOSEK

In [35]:
model_socp = build_problem(filepath,mosek,SOCP);

error: 'filepath' undefined near line 1 column 28


In [36]:
optimize!(model_socp)

parse error:

  syntax error

>>> optimize!(model_socp)
            ^



In [37]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

error: 'model_socp' undefined near line 1 column 22


### Results

In [38]:
models = [model_nl,model_mc,model_sdp,model_socp];

error: 'model_nl' undefined near line 1 column 11


In [39]:
function plot_var(models,variable,title,y_label)
    fig = figure(1)
    ax = fig.add_subplot(1,1,1)
    vars = [value.(model[Symbol(variable)]) for model in models]
    var_length = length(vars[1])
    models_type = ["NL","MC","SDP","SOCP"]
    width = 0.15
    G_labels = ["G"*string(i) for i = 1:var_length]
    xs = 1:var_length
    for (i,var) in enumerate(vars)
        ax.bar(xs.+(i-2.5)*width,var,width,label = models_type[i])
    end
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.set_xticks(xs)
    ax.set_xticklabels(G_labels)
    ax.legend()
    return fig
end

parse error:

  syntax error

>>>     vars = [value.(model[Symbol(variable)]) for model in models]
                            ^

parse error:

  syntax error

>>>     var_length = length(vars[1])
                                ^

models_type = NLMCSDPSOCP
width =  0.15000
parse error:

  syntax error

>>>     G_labels = ["G"*string(i) for i = 1:var_length]
                                     ^

error: 'var_length' undefined near line 1 column 12
parse error:

  syntax error

>>>     for (i,var) in enumerate(vars)
              ^

parse error:

  syntax error

>>>         ax.bar(xs.+(i-2.5)*width,var,width,label = models_type[i])
                                                                  ^

parse error:

  syntax error

>>>     end
          ^

error: 'ax' undefined near line 1 column 5
error: 'ax' undefined near line 1 column 5
error: 'ax' undefined near line 1 column 5
error: 'ax' undefined near line 1 column 5
error: 'ax' undefined near line 1 column 5
parse error:

  synta

In [40]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


In [41]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


## 39 buses

$|\mathcal{B}| = 39 $

$|\mathcal{L}| = 46 $

$|\mathcal{G}| = 10 $

In [42]:
case_test = case_tests[7]

parse error:

  syntax error

>>> case_test = case_tests[7]
                          ^



In [43]:
filepath = joinpath(full_data,case_test[1]*".m")
optimal_value = case_test[2][1]

parse error:

  syntax error

>>> filepath = joinpath(full_data,case_test[1]*".m")
                                           ^

parse error:

  syntax error

>>> optimal_value = case_test[2][1]
                             ^



### Non linear - IPOPT

In [44]:
model_nl = build_problem(filepath,ipopt,NL);

error: 'filepath' undefined near line 1 column 26


In [45]:
optimize!(model_nl)

parse error:

  syntax error

>>> optimize!(model_nl)
            ^



In [46]:
optimality_gap(model_nl,optimal_value)

error: 'model_nl' undefined near line 1 column 16


### McCormick Relaxation - MOSEK

In [47]:
model_mc = build_problem(filepath,mosek,MC);

error: 'filepath' undefined near line 1 column 26


In [48]:
optimize!(model_mc)

parse error:

  syntax error

>>> optimize!(model_mc)
            ^



In [49]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

error: 'model_mc' undefined near line 1 column 22


### Semidefinite Relaxation - MOSEK

In [50]:
model_sdp = build_problem(filepath,mosek,SDP);

error: 'filepath' undefined near line 1 column 27


In [51]:
optimize!(model_sdp)

parse error:

  syntax error

>>> optimize!(model_sdp)
            ^



In [52]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

error: 'model_sdp' undefined near line 1 column 22


### Second Order Cone Relaxation - MOSEK

In [53]:
model_socp = build_problem(filepath,mosek,SOCP);

error: 'filepath' undefined near line 1 column 28


In [54]:
optimize!(model_socp)

parse error:

  syntax error

>>> optimize!(model_socp)
            ^



In [55]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

error: 'model_socp' undefined near line 1 column 22


### Results

In [56]:
models = [model_nl,model_mc,model_sdp,model_socp];

error: 'model_nl' undefined near line 1 column 11


In [57]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


In [58]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


## 162 buses

$|\mathcal{B}| = 162 $

$|\mathcal{L}| = 284 $

$|\mathcal{G}| = 12 $

In [59]:
case_test = case_tests[12]

parse error:

  syntax error

>>> case_test = case_tests[12]
                          ^



In [60]:
filepath = joinpath(full_data,case_test[1]*".m")
optimal_value = case_test[2][1]

parse error:

  syntax error

>>> filepath = joinpath(full_data,case_test[1]*".m")
                                           ^

parse error:

  syntax error

>>> optimal_value = case_test[2][1]
                             ^



### Non linear - IPOPT

In [61]:
model_nl = build_problem(filepath,ipopt,NL);

error: 'filepath' undefined near line 1 column 26


In [62]:
optimize!(model_nl)

parse error:

  syntax error

>>> optimize!(model_nl)
            ^



In [63]:
optimality_gap(model_nl,optimal_value)

error: 'model_nl' undefined near line 1 column 16


### McCormick Relaxation - MOSEK

In [64]:
model_mc = build_problem(filepath,mosek,MC);

error: 'filepath' undefined near line 1 column 26


In [65]:
optimize!(model_mc)

parse error:

  syntax error

>>> optimize!(model_mc)
            ^



In [66]:
optimality_gap_mosek(model_mc,optimal_value,filepath)

error: 'model_mc' undefined near line 1 column 22


### Semidefinite Relaxation - MOSEK

In [67]:
model_sdp = build_problem(filepath,mosek,SDP);

error: 'filepath' undefined near line 1 column 27


In [68]:
optimize!(model_sdp)

parse error:

  syntax error

>>> optimize!(model_sdp)
            ^



In [69]:
optimality_gap_mosek(model_sdp,optimal_value,filepath)

error: 'model_sdp' undefined near line 1 column 22


### Second Order Cone Relaxation - MOSEK

In [70]:
model_socp = build_problem(filepath,mosek,SOCP);

error: 'filepath' undefined near line 1 column 28


In [71]:
optimize!(model_socp)

parse error:

  syntax error

>>> optimize!(model_socp)
            ^



In [72]:
optimality_gap_mosek(model_socp,optimal_value,filepath)

error: 'model_socp' undefined near line 1 column 22


### Results

In [73]:
models = [model_nl,model_mc,model_sdp,model_socp];

error: 'model_nl' undefined near line 1 column 11


In [74]:
plot_var(models,"p_g","Active Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


In [75]:
plot_var(models,"q_g","Reactive Generation","Generation (MW)");

error: 'models' undefined near line 1 column 10


# Benchmark Table

## $|\mathcal{B}| \leq 500 $

In [76]:
tests = case_tests[1:17];

parse error:

  syntax error

>>> tests = case_tests[1:17];
                      ^



### Typical Operating Conditions

In [77]:
cases = benchmark(tests);

error: 'tests' undefined near line 1 column 19


In [78]:
print_cases_table(cases)

error: 'cases' undefined near line 1 column 19


In [79]:
graph_dispersion(cases)

error: 'cases' undefined near line 1 column 18


### Congested Operating Conditions

In [80]:
cases = benchmark(tests,API);

error: 'tests' undefined near line 1 column 19


In [81]:
print_cases_table(cases)

error: 'cases' undefined near line 1 column 19


In [82]:
graph_dispersion(cases)

error: 'cases' undefined near line 1 column 18


### Small Angle Difference Conditions

In [83]:
cases = benchmark(tests,SAD);

error: 'tests' undefined near line 1 column 19


In [84]:
print_cases_table(cases)

error: 'cases' undefined near line 1 column 19


In [85]:
graph_dispersion(cases)

error: 'cases' undefined near line 1 column 18


## $588 \leq |\mathcal{B}| \leq 2000 $

In [86]:
remove = [19]
range = filter(x -> !(x in remove),18:23);

remove =  19
parse error:

  syntax error

>>> range = filter(x -> !(x in remove),18:23);
                      ^



In [87]:
tests = case_tests[range]

parse error:

  syntax error

>>> tests = case_tests[range]
                      ^



### Typical Operating Conditions

In [88]:
cases_no_sdp = benchmark(tests,TYP,false);

error: 'tests' undefined near line 1 column 26


In [89]:
print_cases_no_sdp_table(cases_no_sdp)

error: 'cases_no_sdp' undefined near line 1 column 26


In [90]:
graph_dispersion(cases_no_sdp,sdp=false)

error: 'cases_no_sdp' undefined near line 1 column 18


### Congested Operating Conditions

In [91]:
cases_no_sdp = benchmark(tests,API,false);

error: 'tests' undefined near line 1 column 26


In [92]:
print_cases_no_sdp_table(cases_no_sdp)

error: 'cases_no_sdp' undefined near line 1 column 26


In [93]:
graph_dispersion(cases_no_sdp,sdp=false)

error: 'cases_no_sdp' undefined near line 1 column 18


### Small Angle Difference Conditions

In [94]:
cases_no_sdp = benchmark(tests,SAD,false);

error: 'tests' undefined near line 1 column 26


In [95]:
print_cases_no_sdp_table(cases_no_sdp)

error: 'cases_no_sdp' undefined near line 1 column 26


In [96]:
graph_dispersion(cases_no_sdp,sdp=false)

error: 'cases_no_sdp' undefined near line 1 column 18
